aqui especificar el ETL y hago los requerimientos que me piden en el readme del github... acomodar introduccion

In [145]:
# Importar las bibliotecas necesarias
import numpy as np
import pandas as pd
import json
import ast

In [146]:
#Extraer el dataset
df_movies = pd.read_csv('Dataset/movies_dataset.csv', encoding='utf-8')

C:\Users\evely\AppData\Local\Temp\ipykernel_14400\3148652920.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('Dataset/movies_dataset.csv', encoding='utf-8')


In [147]:
#ojo del video de mateo
print(df_movies.shape)
df_movies.head(1)

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [148]:

# Crear una copia del DataFrame original para no modificarlo directamente
df = df_movies.copy()


In [149]:
# Mostrar solo la columna 'belongs_to_collection'
print(df['belongs_to_collection'])

0        {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                      NaN
2        {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                      NaN
4        {'id': 96871, 'name': 'Father of the Bride Col...
                               ...                        
45461                                                  NaN
45462                                                  NaN
45463                                                  NaN
45464                                                  NaN
45465                                                  NaN
Name: belongs_to_collection, Length: 45466, dtype: object


In [150]:
# Función para desanidar el campo usando ast.literal_eval para la columna belongs_to_collection
def expand_collection(df, belongs_to_collection):
    def safe_literal_eval(x):
        if pd.isna(x) or x == '':
            return {}
        try:
            # Usar ast.literal_eval para convertir la cadena en diccionario
            data = ast.literal_eval(x)
            if isinstance(data, dict):
                return data
            return {}
        except (ValueError, SyntaxError):
            return {}
    
    # Aplicar la función para convertir los valores
    df[belongs_to_collection] = df[belongs_to_collection].apply(safe_literal_eval)
    
    # Crear nuevas columnas a partir del diccionario
    # solo se extrae informacion necesaria de esta columna
    df['idCollection'] = df[belongs_to_collection].apply(lambda x: x.get('id'))
    df['name_collection'] = df[belongs_to_collection].apply(lambda x: x.get('name'))
    
    return df.drop(columns=[belongs_to_collection])

# Aplicar la función
movies = expand_collection(df, 'belongs_to_collection')

# Mostrar las primeras filas para verificar
print(movies.head())


   adult    budget                                             genres  \
0  False  30000000  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  False  65000000  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  False         0  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  False  16000000  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4  False         0                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497                en   
2                                   NaN  15602  tt0113228                en   
3                                   NaN  31357  tt0114885                en   
4                                   NaN  11862  tt0113041                en   

                original_title  \
0                    Toy Story   
1                 

In [151]:
movies.head(1)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,idCollection,name_collection
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,10194.0,Toy Story Collection


Ahora se procede a desanidar las demas columnas 

In [152]:
# Desanidar columnas del dataframe 

# Lista de columnas anidadas a desanidar
columnas_anidadas = ['genres', 'production_companies', 'production_countries', 'spoken_languages']

# Función para desanidar columnas que contienen listas de diccionarios
def desanidar_columna(df, columna):
    # Convertir cadenas de texto en listas/diccionarios si es necesario
    df[columna] = df[columna].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    
    # Desanidar según el tipo de datos en la columna
    if df[columna].apply(lambda x: isinstance(x, list)).any():
        # Para columnas con listas de diccionarios
        desanidado = pd.json_normalize(df[columna].explode().dropna()).reset_index(drop=True)
    else:
        # Para columnas con diccionarios simples
        desanidado = pd.json_normalize(df[columna].dropna()).reset_index(drop=True)
    
    # Renombrar las columnas desanidadas
    desanidado.columns = [f"{columna}_{subcol}" for subcol in desanidado.columns]
    
    # Unir las columnas desanidadas al DataFrame original
    df = df.drop(columns=[columna]).join(desanidado)
    return df

# Desanidar columnas que contienen listas de diccionarios
for columna in ['genres', 'production_companies', 'production_countries', 'spoken_languages']:
    movies = desanidar_columna(movies, columna)


In [153]:
print(movies.shape)

movies.head(1)


(45466, 29)


,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,idCollection,name_collection,genres_id,genres_name,production_companies_name,production_companies_id,production_countries_iso_3166_1,production_countries_name,spoken_languages_iso_639_1,spoken_languages_name
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,10194.0,Toy Story Collection,16,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English


In [ ]:
#Se deben eliminar columnas de las desanidadas que no se usaran ***

In [154]:
# Eliminar las columnas que no serán utilizadas
columns_to_drop = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage',  
                   'production_countries_iso_3166_1', 'spoken_languages_iso_639_1']
movies = movies.drop(columns=columns_to_drop)

In [156]:

# Renombrar algunas columnas
movies = movies.rename(columns={
    'id': 'idMovies',
    'genres_id': 'id_genres',
    'genres_name': 'genres',
    'production_companies_name': 'production_companies',
    'production_companies_id': 'id_production',
    'production_countries_name': 'production_countries',
    'spoken_languages_name': 'spoken_language'

})

# Verificar las primeras filas del DataFrame para asegurarse de que las columnas han sido renombradas
movies.head(1)

,budget,idMovies,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,vote_average,vote_count,idCollection,name_collection,id_genres,genres,production_companies,id_production,production_countries,spoken_language
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,7.7,5415.0,10194.0,Toy Story Collection,16,Animation,Pixar Animation Studios,3.0,United States of America,English


In [157]:
#
print(movies.shape)
movies.head(1)

(45466, 21)


,budget,idMovies,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,vote_average,vote_count,idCollection,name_collection,id_genres,genres,production_companies,id_production,production_countries,spoken_language
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,7.7,5415.0,10194.0,Toy Story Collection,16,Animation,Pixar Animation Studios,3.0,United States of America,English


Transformaciones solicitadas

In [158]:
# Rellenar los valores nulos en las columnas 'revenue' y 'budget' con 0

movies['revenue'] = movies['revenue'].fillna(0)
movies['budget'] = movies['budget'].fillna(0)

# Ver los primeros registros para verificar
print(movies[['revenue', 'budget']].head())


       revenue    budget
0  373554033.0  30000000
1  262797249.0  65000000
2          0.0         0
3   81452156.0  16000000
4   76578911.0         0


In [159]:
# Eliminar las filas donde el campo 'release date' es nulo
movies = movies.dropna(subset=['release_date'])

# Verificar los primeros registros para asegurarse de que se han eliminado las filas nulas
print(movies[['release_date']].head())

  release_date
0   1995-10-30
1   1995-12-15
2   1995-12-22
3   1995-12-22
4   1995-02-10


In [160]:
# Convertir la columna 'release_date' al formato de fecha AAAA-mm-dd
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce', format='%Y-%m-%d')

# Crear la columna 'release_year' extrayendo el año de 'release_date' y convirtiendo a formato de fecha con frecuencia anual
movies['release_year'] = movies['release_date'].dt.to_period('Y')

# Opcional: Eliminar filas con 'release_year' nulo
movies = movies.dropna(subset=['release_year'])

# Verificar los primeros registros para asegurarse de que las fechas están formateadas y el año ha sido extraído
print(movies[['release_date', 'release_year']].head())


  release_date release_year
0   1995-10-30         1995
1   1995-12-15         1995
2   1995-12-22         1995
3   1995-12-22         1995
4   1995-02-10         1995


In [161]:
#Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, 
# dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0

# Asegurarse de que las columnas 'revenue' y 'budget' sean numéricas
movies['revenue'] = pd.to_numeric(movies['revenue'], errors='coerce')
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce')

# Crear la columna 'return' calculando revenue / budget
# Si el budget es 0 o NaN, el retorno también debe ser 0
movies['return'] = movies.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 else 0, axis=1)

# Verificar los primeros registros para asegurarse de que el cálculo es correcto
print(movies[['revenue', 'budget', 'return']].head())


       revenue    budget     return
0  373554033.0  30000000  12.451801
1  262797249.0  65000000   4.043035
2          0.0         0   0.000000
3   81452156.0  16000000   5.090760
4   76578911.0         0   0.000000


Realizar otras modificaciones necesarias

In [163]:
# Verifica los primeros registros para asegurarte de que la desanidación se realizó correctamente****
print(movies[['idCollection','name_collection']].head())

   idCollection                 name_collection
0       10194.0            Toy Story Collection
1           NaN                            None
2      119050.0       Grumpy Old Men Collection
3           NaN                            None
4       96871.0  Father of the Bride Collection


In [164]:
#Verificacion de los tipos de datos
print(movies.dtypes)

budget                           int64
idMovies                        object
original_language               object
overview                        object
popularity                      object
release_date            datetime64[ns]
revenue                        float64
runtime                        float64
status                          object
tagline                         object
title                           object
vote_average                   float64
vote_count                     float64
idCollection                   float64
name_collection                 object
id_genres                        int64
genres                          object
production_companies            object
id_production                  float64
production_countries            object
spoken_language                 object
release_year             period[Y-DEC]
return                         float64
dtype: object


In [165]:
# Modificación de idMovies
# Convertir la columna 'id' a entero
movies['idMovies'] = movies['idMovies'].astype(int)


# Modificación de idProduccion
# Llenar los valores nulos en 'idProduccion' con 0
movies['id_production'] = movies['id_production'].fillna(0)

# Convertir la columna 'idProduccion' a entero
movies['id_production'] = movies['id_production'].astype(int)


In [166]:
# Convertir 'popularity' a float64
movies['popularity'] = movies['popularity'].astype(float)

# Verificar los tipos de datos después de las conversiones
print(movies.dtypes)

budget                           int64
idMovies                         int32
original_language               object
overview                        object
popularity                     float64
release_date            datetime64[ns]
revenue                        float64
runtime                        float64
status                          object
tagline                         object
title                           object
vote_average                   float64
vote_count                     float64
idCollection                   float64
name_collection                 object
id_genres                        int64
genres                          object
production_companies            object
id_production                    int32
production_countries            object
spoken_language                 object
release_year             period[Y-DEC]
return                         float64
dtype: object


Verificacion de Campos Nulos

In [167]:
#valores nulos
movies.isnull().sum()

budget                      0
idMovies                    0
original_language          11
overview                  941
popularity                  0
release_date                0
revenue                     0
runtime                   246
status                     80
tagline                 24978
title                       0
vote_average                0
vote_count                  0
idCollection            40888
name_collection         40888
id_genres                   0
genres                      0
production_companies        1
id_production               0
production_countries        3
spoken_language             0
release_year                0
return                      0
dtype: int64

In [169]:
print(movies['name_collection'].unique())

['Toy Story Collection' None 'Grumpy Old Men Collection' ...
 'Ducobu Collection' 'Mister Blot Collection' 'Red Lotus Collection']


Resumen: Eliminación de la Columna coleccion_de_peliculas
La columna coleccion_de_peliculas fue eliminada del conjunto de datos debido a su bajo nivel de cobertura y relevancia para el proyecto. Debido a su alta proporción de Valores Nulos: De las 45,466 películas en el conjunto de datos, 40,888 (aproximadamente el 90%).Esto indica que la mayoría de las películas no pertenecen a una franquicia o colección identificada, lo que limita significativamente su utilidad como característica predictiva para el sistema de recomendación.


In [170]:
# Eliminar las columnas que no serán utilizadas
columns_to_drop = ['idCollection', 'name_collection']
movies = movies.drop(columns=columns_to_drop)

In [171]:
#valores nulos
movies.isnull().sum()

budget                      0
idMovies                    0
original_language          11
overview                  941
popularity                  0
release_date                0
revenue                     0
runtime                   246
status                     80
tagline                 24978
title                       0
vote_average                0
vote_count                  0
id_genres                   0
genres                      0
production_companies        1
id_production               0
production_countries        3
spoken_language             0
release_year                0
return                      0
dtype: int64

In [172]:
# Diccionario con columnas y valores de relleno
fill_values = {
    
    'original_language': 'unknown',
    'overview': 'no summary',
    'tagline': 'no summary',
    'status': 'unknown',
    'compañiaProduccion': 'unknown',
    'paisProduccion': 'unknown',
    'runtime': 0
    
}

# Rellenar los valores nulos en las columnas especificadas usando un diccionario
movies.fillna(value=fill_values, inplace=True)

In [176]:
movies.shape


(45376, 21)

In [173]:
#valores nulos
movies.isnull().sum()

budget                  0
idMovies                0
original_language       0
overview                0
popularity              0
release_date            0
revenue                 0
runtime                 0
status                  0
tagline                 0
title                   0
vote_average            0
vote_count              0
id_genres               0
genres                  0
production_companies    1
id_production           0
production_countries    3
spoken_language         0
release_year            0
return                  0
dtype: int64

****OJO PONER TEXTO DEL IDIOMA

In [174]:
import re

def contains_non_latin(text):
    # Verifica si hay caracteres que no sean latinos (ASCII)
    if pd.isnull(text):
        return False
    return bool(re.search('[^\u0000-\u007F]+', str(text)))

# Aplicar la función a las columnas
for col in movies.columns:
    if movies[col].apply(contains_non_latin).any():
        print(f"Columna con caracteres no latinos: {col}")


Columna con caracteres no latinos: overview
Columna con caracteres no latinos: tagline
Columna con caracteres no latinos: title
Columna con caracteres no latinos: production_companies
Columna con caracteres no latinos: spoken_language


overview', 'tagline', 'compañiaProduccion', 'spoken_language'

"Durante la fase de ETL, se identificaron columnas con caracteres no latinos en el dataset, como title, overview, tagline, compañiaProduccion, y spoken_language. Estas columnas se traducirán al inglés usando una herramienta automática de traducción para garantizar uniformidad en el idioma. Este paso es crucial para asegurar que los textos puedan ser procesados correctamente en el modelo de recomendación y evitar problemas derivados de la diferencia de idiomas."

ojo ****"Durante la fase de ETL, se han aplicado varias técnicas de procesamiento a las columnas textuales del dataset para optimizar el modelado de recomendación de películas. Estas técnicas incluyen la normalización del texto, extracción de características mediante TF-IDF, obtención de embeddings de palabras, y codificación de categorías. La normalización asegura la uniformidad en el análisis, mientras que las técnicas de extracción de características y embeddings permiten una representación más rica y útil de los datos textuales."

In [ ]:
from polyglot.detect import Detector
import pandas as pd

def filter_text_by_polyglot(df, columns, language='en'):
    def is_language(text):
        if pd.isna(text) or text.strip() == '':
            return False
        try:
            detector = Detector(text)
            return detector.language.code == language
        except Exception as e:
            print(f"Error detectando idioma: {e}")
            return False

    for column in columns:
        df[column] = df[column].apply(lambda x: x if is_language(x) else None)
    return df

# Ejemplo de uso
columns_to_check = ['title', 'overview', 'tagline', 'compañiaProduccion', 'spoken_language']
df = filter_text_by_polyglot(df, columns_to_check)




In [175]:

# Exportar los cambios realizados 
movies.to_csv('movies_transformado.csv', index=False)
